In [1]:
import netCDF4 as nc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import pyproj
from scipy.interpolate import griddata
import cftime

In [ ]:
home_path = os.path.expanduser("~")

path = '/DataFiles'
eraPrecip = xr.open_dataset(home_path + path + "/pr_sfc_Amon_ERA20C_190001-201012.nc")

path2 = '/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents - MacBook Air/Python/Ice Cores/data/model/ccsm4_last_millenium'

allEra = xr.open_dataset(home_path + path2 + '/era5total.nc')
eraPrecip = xr.open_dataset(home_path + path2 + '/era5total.nc')

In [ ]:
# pr has units kg/m^2/s = mm/s
pr = eraPrecip["pr"]


lats = eraPrecip["lat"].values
lons = eraPrecip["lon"].values

# Compute number of days in each month (xarray provides days_in_month)
days_in_month = pr["time"].dt.days_in_month
seconds_in_month = days_in_month * 24 * 3600

# Convert monthly mean rate (mm/s) → monthly total (mm)
pr_monthly_total = pr * seconds_in_month

# Restrict to 1979–2000
#pr_sel = pr_monthly_total.sel(time=slice("1979-01-01", "2000-12-31"))
pr_sel = pr_monthly_total.sel(time=slice("1980-01-01", "2000-12-31"))

# Group by year and sum over months
pr_annual = pr_sel.groupby("time.year").sum("time")

# pr_annual is now (year, lat, lon) with units mm/year
print(pr_annual.shape)

KeyError: 'lat'

pr_annual stores the 

In [4]:
prCoords = pd.read_csv(home_path + path + "/AccumCoresCoords.csv", header=None)
coresLat = prCoords.iloc[:, 2].values
coresLonFirst = prCoords.iloc[:, 3].values
coresLon = (coresLonFirst + 360) % 360

In [5]:
lat_idx = []
lon_idx = []
grid_lats = []
grid_lons = []

for la, lo in zip(coresLat, coresLon):
    pt = eraPrecip.sel(lat=la, lon=lo, method="nearest")

    # Get matched coordinates
    glat = float(pt.lat.values)
    glon = float(pt.lon.values)

    # Find indices in the original arrays
    i_lat = np.where(lats == glat)[0][0]
    i_lon = np.where(lons == glon)[0][0]

    grid_lats.append(glat)
    grid_lons.append(glon)
    lat_idx.append(i_lat)
    lon_idx.append(i_lon)

# --- Add results back to DataFrame ---
prCoords["lat_idx"] = lat_idx
prCoords["lon_idx"] = lon_idx
prCoords["grid_lat"] = grid_lats
prCoords["grid_lon"] = grid_lons

print(prCoords.head())

                       0                       1        2       3  lat_idx  \
0  vrs-13 (vostok stack)  Vostok composite VRS13 -78.4700  106.83      168   
1              B31 3.43W           B31Site DML07 -75.5800   -3.43      166   
2           B32 0.00667E           B32Site DML05 -75.0000   -0.01      165   
3            B33 6.4983E           B33Site DML17 -75.1700    6.50      165   
4              FB96DML01               FB96DML01 -74.8583   -2.55      165   

   lon_idx  grid_lat  grid_lon  
0      107     -78.0     107.0  
1      357     -76.0     357.0  
2      359     -75.0     359.0  
3        7     -75.0       7.0  
4      357     -75.0     357.0  


In [6]:
print(prCoords)

                        0                       1        2       3  lat_idx  \
0   vrs-13 (vostok stack)  Vostok composite VRS13 -78.4700  106.83      168   
1               B31 3.43W           B31Site DML07 -75.5800   -3.43      166   
2            B32 0.00667E           B32Site DML05 -75.0000   -0.01      165   
3             B33 6.4983E           B33Site DML17 -75.1700    6.50      165   
4               FB96DML01               FB96DML01 -74.8583   -2.55      165   
..                    ...                     ...      ...     ...      ...   
80      Mount Brown South       Mount Brown South -69.1100   86.31      159   
81              MBS-Alpha               MBS-Alpha -69.1100   86.31      159   
82              MBS-Bravo               MBS-Bravo -69.1100   86.31      159   
83            MBS-Charlie             MBS-Charlie -69.1100   86.31      159   
84                    B40                     B40 -75.0000    0.07      165   

    lon_idx  grid_lat  grid_lon  
0       107     -

In [7]:
accumData = pd.read_csv(home_path + path + "/AccumCoresData.csv", header=None)
print(accumData)

              0                      1          2             3            4   \
0    Site number                      1          2             3            4   
1           Year  vrs-13 (vostok stack)  B31 3.43W  B32 0.00667E  B33 6.4983E   
2           2020                    NaN        NaN           NaN          NaN   
3           2019                    NaN        NaN           NaN          NaN   
4           2018                    NaN        NaN           NaN          NaN   
..           ...                    ...        ...           ...          ...   
218         1804                  39.93       69.6          69.5         31.7   
219         1803                  17.04       75.7            78         41.1   
220         1802                  26.64       66.2          63.3         45.1   
221         1801                   5.79         52          62.9         44.6   
222         1800                  30.37         79          80.8         45.7   

            5          6   

In [8]:
print(prCoords)

                        0                       1        2       3  lat_idx  \
0   vrs-13 (vostok stack)  Vostok composite VRS13 -78.4700  106.83      168   
1               B31 3.43W           B31Site DML07 -75.5800   -3.43      166   
2            B32 0.00667E           B32Site DML05 -75.0000   -0.01      165   
3             B33 6.4983E           B33Site DML17 -75.1700    6.50      165   
4               FB96DML01               FB96DML01 -74.8583   -2.55      165   
..                    ...                     ...      ...     ...      ...   
80      Mount Brown South       Mount Brown South -69.1100   86.31      159   
81              MBS-Alpha               MBS-Alpha -69.1100   86.31      159   
82              MBS-Bravo               MBS-Bravo -69.1100   86.31      159   
83            MBS-Charlie             MBS-Charlie -69.1100   86.31      159   
84                    B40                     B40 -75.0000    0.07      165   

    lon_idx  grid_lat  grid_lon  
0       107     -

In [9]:
print(pr_annual)

<xarray.DataArray (year: 22, lat: 181, lon: 360)> Size: 11MB
array([[[160.82958513, 160.82958513, 160.82958513, ..., 160.82958513,
         160.82958513, 160.82958513],
        [159.87787332, 159.93614139, 160.04296619, ..., 159.9167187 ,
         159.90215168, 159.85359496],
        [170.66232198, 170.90024994, 171.14303356, ..., 169.46782654,
         169.86599169, 170.26415684],
        ...,
        [ 45.54135254,  45.26943488,  44.90525944, ...,  46.83781711,
          46.42022927,  45.99778576],
        [ 56.50060542,  56.25782179,  56.05873922, ...,  57.04929641,
          56.81622413,  56.66569828],
        [ 65.36220776,  65.36220776,  65.36220776, ...,  65.36220776,
          65.36220776,  65.36220776]],

       [[163.485638  , 163.485638  , 163.485638  , ..., 163.485638  ,
         163.485638  , 163.485638  ],
        [160.31973952, 160.61107987, 160.88299753, ..., 160.23233742,
         160.30031683, 160.30031683],
        [168.70548596, 168.82687777, 168.82687777, ..., 168.

In [15]:
print(pr_annual[:, -25, 0])

<xarray.DataArray (year: 22)> Size: 176B
array([458.84648592, 388.81312119, 478.05067072, 445.35257199,
       501.31419776, 467.26136639, 473.93306043, 531.24456316,
       417.68009431, 461.10922931, 451.70379164, 622.48750544,
       520.19790818, 517.15825718, 476.27835025, 532.51189368,
       538.02793766, 580.51992787, 563.06864084, 568.40502493,
       505.669736  , 471.04393528])
Coordinates:
    lat      float32 4B -66.0
    lon      float32 4B 0.0
  * year     (year) int64 176B 1979 1980 1981 1982 1983 ... 1997 1998 1999 2000


In [20]:
era5Metrics = np.array(pd.read_csv(home_path + path + '/ERA5Metrics.csv'))

era5_accum_ais = era5Metrics[:, 1]
era5_accum_wais = era5Metrics[:, 2]
era5_accum_eais = era5Metrics[:, 3]
era5_temp_ais = era5Metrics[:, 4]
era5_temp_wais = era5Metrics[:, 5]
era5_temp_eais = era5Metrics[:, 6]